In [2]:
import sys
import os
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import xlrd
import openpyxl

sys.path.append("..")
from config import *
from lib import *

# 对已经过大概文字处理的excel进行补丁操作，修正等

trans_merge_filename = os.path.join(HOME_DIR,TEMP_DIR,'Poe_tempcustom.xlsx')
patch_filename = os.path.join(STORAGE_DIR,'Poe_patch_table.xlsx')

xmdf = pd.read_excel(trans_merge_filename).fillna('')
tmdf = xmdf.copy()

In [5]:
tmdf.ix['game\\gui_299']

Name                                           game\gui
ID                                                  299
DefaultText         You have gained an item ({1}): {0}.
FemaleText                                             
[ali]DefaultText                    你 获 得 物 品 ： {1}x{0}
[ali]FemaleText                                        
Custom                                                 
Custom_female                                          
merged                                   你获得物品： {1}x{0}
merged_female                                          
Name: game\gui_299, dtype: object

In [8]:
def patch_result_list(patch_filename):
    # 读取替换表，并依照表格顺序，依次转化为df并有序添加到array中
    book = openpyxl.load_workbook(patch_filename)

    tr_list = {} # 无序
    tr2_list = [] # 有序 [{'name':工作表名,'data':工作表DF数据}]

    for sheet in book:
        print(sheet.title)
        _sn = sheet.title
        pr = pd.read_excel(patch_filename,sheetname=_sn).fillna('')
        tr2_list.append({'name':_sn,'data':pr})
        tr_list[_sn]=pr

    print('替换表读取完毕')
    return tr2_list

# tr2_list = patch_result_list(patch_filename)

In [29]:
# 查看序列
# ['fix', '人名', '名词', '地名', '怪物', '机制', '武器', '物品', '特有', '状态', '神祇']

# for x in tr2_list:
#     print(x['name'])
    
ffx = os.path.join(STORAGE_DIR, 'Tyr_guifix.xlsx')
patch_list = pd.read_excel(ffx).fillna('')
try:
    patch_list = patch_list.set_index('index')
except:
    patch_list.index = [patch_list['Name'] + '_' + patch_list['ID'].astype('str')]
    patch_list.index.name = 'index'

In [47]:
# patch_list[['DefaultText','Custom']].replace('0', 'H')
patch_list[['DefaultText','Custom']].replace(to_replace='{H}', value='0')
# patch_list

,DefaultText,Custom
index,,
game\gui_2782,Act {0},第 {0} 幕
game\gui_153,"Day {7}, Hour {2}",第 {7} 天， {2} 时
game\gui_1493,"({0}d, {1}h, {2}m)",({0}天，{1}时，{2}分)
game\gui_1576,{0} {1} - {2},{0} {1} - {2}
game\gui_385,,第 {0} 章，{1}
game\gui_60,{0} added: '{1}',{0}已添加：『{1}』
game\gui_61,{0} updated: '{1}',{0}已更新：『{1}』
game\gui_62,{0} completed: '{1}',{0}已完成：『{1}』
game\gui_150,{0} failed: '{1}',{0}已失败：『{1}』


In [11]:
# Patch way #2:
# 先查询DefaultText列，找出含有对应英文的行
# 遍历结果行，依照替换表对应的词，替换已翻译中文
# 使用df.ix[] 方式，直接替换大的df表上的数据...并不返回大表..

def patch2(df):
    tmdf = df.copy()
    # 要处理的列
    m_key = 'merged'
    f_key = 'merged_female'

    countrep = 0

    # data is DataFrame
    def _fix(data):
        print('=== Strat fix')
        localcount = 0
        try:
            data.set_index('index')
        except:
            data.index = [data['Name'] + '_' + data['ID'].astype('str')]
            data.index.name = 'index'

        for index, row in data.iterrows():
            if row['Chs'] == '' or row['Chs'] is None:
                continue
            else:
                im = index
                #  im = tmdf[(tmdf['Name']==row['Name']) & (tmdf['ID']==row['ID'])].index
                tmdf.ix[im, m_key] = row['Chs']
                localcount += 1
        print('{s} row patched ===End fix'.format(s = localcount))
        return data


    # pre is Series
    def tuidao(pre):
        chs = pre['Chs']
        eng = pre['Eng']
        rep = pre['Rep'].split(',')
        r_count = 0
        if chs == '' or chs is None:
            return

        items = tmdf[tmdf['DefaultText'].str.contains(eng, case=False)]

        for i,row in items.iterrows():   
            if len(rep) < 2:
                row[m_key] = row[m_key].replace(rep[0],chs)
                if row['FemaleText'] != '':
                    # 使男女用一样的对话，不区分 - -' 
                    row[f_key] = row[m_key]
            else:
                for k in rep:
                    row[m_key] = row[m_key].replace(k,chs)
                    if row['FemaleText'] != '':
                        row[f_key] = row[m_key]
            r_count +=1
            tmdf.ix[i,[m_key,f_key]] = [row[m_key],row[f_key]]

        print('{co} replaced: {e} {s} ==> {b}'.format(co = r_count, e=eng, s = rep, b = chs))

    def _map(data):
        print('===strat replace')
        for index, row in data.iterrows():
            if row['Rep'] == '' or row['Rep'] is None or row['Chs'] == '' or row['Chs'] is None:
                continue
            else:
                tuidao(row)
        print('===patch end')
        return data
    
    
    def _start():
        tr2_list = patch_result_list(patch_filename)
        for x in tr2_list:
            if x['name'] == 'fix':
                _fix(x['data'])
            else:
#                 _map(x['data'])
                print('pass name replace')
        return tmdf
    
    return _start()

tmdf = patch2(tmdf)

名词
地名
物品
神祇
人名
状态
机制
特有
武器
怪物
fix
替换表读取完毕
pass name replace
pass name replace
pass name replace
pass name replace
pass name replace
pass name replace
pass name replace
pass name replace
pass name replace
pass name replace
=== Strat fix
74 row patched ===End fix


In [12]:
# tmdf.ix[16741]
tmdf.ix['game\\gui_299']

Name                                           game\gui
ID                                                  299
DefaultText         You have gained an item ({1}): {0}.
FemaleText                                             
[ali]DefaultText                    你 获 得 物 品 ： {1}x{0}
[ali]FemaleText                                        
Custom                                                 
Custom_female                                          
merged                                    你获得物品：{1}x{0}
merged_female                                          
Name: game\gui_299, dtype: object

In [ ]:
# 去重并部分导出
tmdf = tmdf.drop_duplicates()
tmdf[[m_key,f_key]].to_excel(os.path.join(HOME_DIR,OUTPUT_DIR,'Poe_final.xlsx'))

In [ ]:
varfix = tmdf[tmdf['DefaultText'].str.contains('\{*\}')]
varfix.to_excel(os.path.join(HOME_DIR,OUTPUT_DIR,'var.xlsx'),index=False)

In [ ]:
baike = tmdf[tmdf['Name'].str.contains('cyclopedia')]
baike.to_excel(os.path.join(HOME_DIR,OUTPUT_DIR,'cyclopedia.xlsx'),index=False)

In [ ]:
quests = tmdf[(tmdf['Name'].str.contains('quests')) & (tmdf['ID']==0)]
quests

In [ ]:

merge_patched_file = os.path.join(HOME_DIR,OUTPUT_DIR,'merge_patched.xlsx')
mpf = pd.read_excel(merge_patched_file).fillna('')

In [ ]:
abi = pd.read_excel( os.path.join(STORAGE_DIR,'abilities.xlsx')).fillna('')
abi

In [ ]:
sp = pd.merge(abi, mpf, on=['Name','DefaultText'],how='left')
sp

In [ ]:
# .to_excel(os.path.join(STORAGE_DIR,'abilities2.xlsx'),index=False)
sp[['Name','ID','DefaultText','Custom_y']].to_excel(os.path.join(STORAGE_DIR,'abilities2.xlsx'),index=False)